In [1]:
from CS2_bo_functions import LHS_Design
from CS2_bo_functions import create_y_data
from CS2_bo_functions import calc_y_exp
from CS2_bo_functions import test_train_split
from CS2_bo_functions import create_sse_data
from scipy.stats import qmc
import torch
import numpy as np
import pandas as pd
import os

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#LHS Design in python - May switch code to use this instead for conciseness
emulator = True
seed = 9
LHS_Points =20
d = 4
sampler = qmc.LatinHypercube(d=d, seed = seed)
sample = sampler.random(n=LHS_Points)
l_bounds =  [-2, -2, -10 ,-2]
u_bounds =  [0, 0 , 0, 2 ]
LHS = qmc.scale(sample, l_bounds, u_bounds)

In [3]:
#LHS for experimental data points
exp_LHS_Points = 25 #0 , 10, 15, 20, or 25 for 4, 8, 12, 16, and 24 params
exp_d = 2
exp_sampler = qmc.LatinHypercube(d=exp_d, seed = seed)
exp_sample = exp_sampler.random(n=exp_LHS_Points)

exp_l_bounds =  [-1.5, -0.5]
exp_u_bounds =  [1 , 2]
exp_LHS = qmc.scale(exp_sample, exp_l_bounds, exp_u_bounds)
print(exp_LHS)

[[ 0.51297508  1.97131828]
 [-1.46031482  0.92224659]
 [-0.27160746  1.70846199]
 [ 0.21396064  0.70817624]
 [-0.00265877  0.0562752 ]
 [-0.84849443  0.69348458]
 [ 0.79943739  1.61693786]
 [ 0.60166978 -0.37846099]
 [-0.93156035  0.52946663]
 [-0.42991811  1.02592515]
 [-1.02797472  1.42174091]
 [-0.69877407  1.30137615]
 [-0.38828626  1.80871899]
 [ 0.82917931  0.84455031]
 [-1.19232382  0.39102692]
 [ 0.36298685  1.13940611]
 [ 0.05222701  0.42071421]
 [ 0.48049136  0.29377465]
 [-1.21265184 -0.29627816]
 [-0.71327745  1.57739017]
 [-1.33770564 -0.16234216]
 [ 0.92179039  1.29213511]
 [-0.59312375  0.12372083]
 [-0.13209641 -0.4189332 ]
 [ 0.14040471 -0.05946892]]


In [4]:
#Creates Experimental data and saves to a csv
Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])

Xexp = np.array([[-0.558,-0.050,0.623,-0.82, 0.22],
                 [1.442,0.467,0.028,0.62,0.30]])

Xexp = np.vstack((exp_LHS, Xexp.T))
print(Xexp[0])
print(Xexp.shape)


[0.51297508 1.97131828]
(30, 2)


In [5]:
n = Xexp.shape[0]
print(n)
noise_std = 0.01
Yexp = calc_y_exp(Constants, Xexp, noise_std)
print(Yexp)

df = pd.DataFrame({'X1exp': Xexp[:, 0], 'X2exp': Xexp[:, 1], 'Yexp':Yexp})
print(df)

path = 'Input_CSVs/Exp_Data'
if not os.path.exists(path):
    os.makedirs(path)
df.to_csv(os.path.join(path,'n='+str(n)+'.csv'))

30
[ 338.41622638   -5.7885826  -112.04546332  -26.53201777  -52.95078054
  -42.47984572  367.64098133   50.07326647  -35.81198952  -12.73285152
  -25.16684798 -139.20600027  -81.83556608  118.32331828  -12.22012788
   61.06790996  -78.95875122  -63.62694608   58.55587756  -79.29791827
   51.95382584  295.79490431   -7.35912456   40.06959465  -56.8790201
 -146.69947812  -80.76773816 -108.16663897  -40.67072465  -72.22508357]
       X1exp     X2exp        Yexp
0   0.512975  1.971318  338.416226
1  -1.460315  0.922247   -5.788583
2  -0.271607  1.708462 -112.045463
3   0.213961  0.708176  -26.532018
4  -0.002659  0.056275  -52.950781
5  -0.848494  0.693485  -42.479846
6   0.799437  1.616938  367.640981
7   0.601670 -0.378461   50.073266
8  -0.931560  0.529467  -35.811990
9  -0.429918  1.025925  -12.732852
10 -1.027975  1.421741  -25.166848
11 -0.698774  1.301376 -139.206000
12 -0.388286  1.808719  -81.835566
13  0.829179  0.844550  118.323318
14 -1.192324  0.391027  -12.220128
15  0.36298

In [6]:
#Array for Standard Approach
all_p_standard = pd.DataFrame(LHS, columns = ["a1", "a2", "a3", "a4"])


#Array for Emulator approach
df = all_p_standard.iloc[np.repeat(np.arange(len(all_p_standard)), n)]
df = df.reset_index(drop = True)
Xexp_repeat = np.repeat(Xexp, LHS_Points, axis = 0).reshape(Xexp.shape[1],-1)
# print(Xexp_repeat)
#Add Xexp 
for i in range(Xexp.shape[1]):
    df['X'+str(i+1)+'exp'] = Xexp_repeat[i]    
all_p_emulator = df
# print(all_p_emulator)
# print(all_p_standard)


In [7]:
y_data = create_y_data(all_p_emulator, Constants)
sse_data = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "obj")
# print(y_data)
# print(sse_data)

In [8]:
#Create Training Data and Testing Data
#Set noise parameters and load csv file to generate training data
# all_p = LHS_Design("LHS_Toy_3_Input.csv")

#Create y or SSE data
#Save data
if emulator == False:
    all_2_data = all_p_standard.copy()
    all_2_in_y = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "obj")
    all_2_data['Yexp'] = all_2_in_y
    
    all_2_data_ln_obj = all_p_standard.copy()
    all_2_in_y_ln_obj = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "LN_obj")
    all_2_data_ln_obj['Yexp'] = all_2_in_y_ln_obj
    
    titles_all = np.array(['all_st_data','all_st_ln_obj_data'])
    len_data = str(len(all_2_data))
    all_array = [all_2_data,all_2_data_ln_obj]

else:
    all_3_data = all_p_emulator.copy()
    all_3_in_y = create_y_data(all_p_emulator, Constants)
    all_3_data['Yexp'] = all_3_in_y
    
    titles_all = np.array(['all_emul_data'])
    len_data = str(len(all_3_data))
    all_array = [all_3_data]
    
# print(all_array)

# Save Train Data/Test Data
for i in range(len(titles_all)):
    path = 'Input_CSVs/Train_Data/'+titles_all[i]+"/"
    if not os.path.exists(path):
        os.makedirs(path)
    all_array[i].to_csv(os.path.join(path,'len_data='+len_data)+'.csv')